In [4]:
# from google.colab import drive
# drive.mount('/content/drive')
!pip install eli5

In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance  

from ast import literal_eval
from tqdm import tqdm_notebook

In [8]:
cd drive/My\ Drive/Colab Notebooks/Matrix_DW

/content/drive/My Drive/Colab Notebooks/Matrix_DW


In [10]:
ls data

men_shoes.csv  shoes_prices.csv  valid_men_shoes.csv


In [0]:
df = pd.read_csv('data/valid_men_shoes.csv', low_memory=False)

In [13]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model=DecisionTreeRegressor(max_depth=5)):
  '''model przekazalismy jako parametr zeby mieć mozliwosc wyboru jakiego chcemy uzyc'''
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [23]:
df['brand_category'] = df['brand'].factorize()[0]
run_model(['brand_category'])

(-58.38655694633361, 4.223555478221712)

In [25]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_category'], model)

(-57.47223572384038, 4.328288468270897)

In [26]:
df['brand_category'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_category'], model)

(-57.31783843165656, 4.181246596160967)

In [34]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [37]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict) # to samo jakbym do str_dict przypisal listę bez ciapkow

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
# pierwszy etap
def parse_features(x):
  if str(x) == 'nan': return []
  return literal_eval(x.replace('\\"', '"')) # replace jest dlatego ze jest dziwna wartosc i chcemy to poprawic

df['features_parsed'] = df.features.map(parse_features)

In [45]:
df['features_parsed'].head().values

array([list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Shoe Size', 'value': ['M']}, {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, {'key': 'Color', 'value': ['Multicolor']}, {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, {'key': 'Brand', 'value': ['Josmo']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in pounds)', 'value': ['0.45']}, {'key': 'Condition', 'value': ['New']}, {'key': 'Brand', 'value': ['SERVUS BY HONEYWELL']}, {'key': 'manufacturer_part_number', 'value': ['ZSR101BLMLG']}]),
       list([{'key': 'Gender', 'value': ['Men']}, {'key': 'Color', 'value': ['Black']}, {'key': 'Shipping Weight (in

In [47]:
# powyżej w pierwszym wierszu jet lista słownikow a my chcemy z tej listy mieć tylko jeden slownik:
{'Gender': 'Men',
 'Shoe Size': 'M'}

{'Gender': 'Men', 'Shoe Size': 'M'}

In [50]:
# drugi etap
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict
  features = literal_eval(x.replace('\\"', '"'))

  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key] = value
  
  return output_dict

df['features_parsed'] = df.features.map(parse_features)
df['features_parsed'].head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [51]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [60]:
df['features_parsed']

0        {'gender': 'men', 'shoe size': 'm', 'shoe cate...
1        {'gender': 'men', 'shoe size': 'm', 'shoe cate...
2        {'gender': 'men', 'color': 'black', 'shipping ...
3        {'gender': 'men', 'color': 'black', 'shipping ...
4        {'gender': 'men', 'color': 'black', 'shipping ...
                               ...                        
18275    {'style': 'desert boots', 'pattern': 'solid', ...
18276    {'shoe size': '9.5', 'gender': 'men', 'shoe ca...
18277    {'gender': 'unisex', 'frame style': 'wrap', 'p...
18278    {'gender': 'unisex', 'frame style': 'wrap', 'p...
18279    {'gender': 'unisex', 'frame style': 'wrap', 'p...
Name: features_parsed, Length: 18280, dtype: object

In [69]:
df['features_parsed'].map(lambda x: x.keys())

0        (gender, shoe size, shoe category, color, manu...
1        (gender, shoe size, shoe category, color, manu...
2        (gender, color, shipping weight (in pounds), c...
3        (gender, color, shipping weight (in pounds), c...
4        (gender, color, shipping weight (in pounds), c...
                               ...                        
18275                          (style, pattern, condition)
18276     (shoe size, gender, shoe category, brand, color)
18277       (gender, frame style, polarized lenses, color)
18278       (gender, frame style, polarized lenses, color)
18279       (gender, frame style, polarized lenses, color)
Name: features_parsed, Length: 18280, dtype: object

In [70]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [88]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'featreinforced knee', 'featvideo game platform', 'featjewelry setting',
       'featinternational shipping', 'featgenre', 'feateye size:',
       'featdisplay', 'featcrystal', 'featremovable hood', 'featband color'],
      dtype='object', length=526)

In [92]:
df[df['featathlete'].isnull()].shape

(18272, 526)

In [94]:
df[False == df['featathlete'].isnull()].shape[0]

8

In [96]:
df[False == df['featathlete'].isnull()].shape[0] / df.shape[0] * 100

0.0437636761487965

In [103]:
keys_stat = {}
for key in keys:
  keys_stat[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

{k:v for k,v in keys_stat.items() if v >30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['featbrand_category'] = df['featbrand'].factorize()[0]
df['featcolor_category'] = df['featcolor'].factorize()[0]
df['featgender_category'] = df['featgender'].factorize()[0]
df['featmanufacturer part number_category'] = df['featmanufacturer part number'].factorize()[0]
df['featmaterial_category'] = df['featmaterial'].factorize()[0]

df['featsport_category'] = df['featsport'].factorize()[0]
df['featstyle_category'] = df['featstyle'].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_category'] = df[get_name_feat(key)].factorize()[0]

In [118]:
df[df.brand == df.featbrand].shape

(8846, 531)

In [119]:
df['brand'] = df['brand'].map(lambda x: str(x).lower()) # normalizacja, czyli dane w brand zamienione na mala litere
df[df.brand == df.featbrand][['brand', 'featbrand']].shape

(8846, 2)

In [120]:
 model = RandomForestRegressor(max_depth=5, n_estimators=100)
 run_model(['brand_category'], model)

(-57.326013950457096, 4.225466463375625)

In [199]:
feats_category = [x for x in df.columns if 'category' in x]
feats_category

['brand_category',
 'featshoe category',
 'featclothing category',
 'featbrand_category',
 'featcolor_category',
 'featgender_category',
 'featmanufacturer part number_category',
 'featmaterial_category',
 'featsport_category',
 'featstyle_category',
 'featdesigned for_category',
 'featband length (in.)_category',
 'featcushioning_category',
 'featcountry of origin assembly:_category',
 'featclothing type_category',
 'featreinforced pockets_category',
 'featprotects against_category',
 'featproduct in inches (l x w x h)_category',
 'featitem width_category',
 'featfits models_category',
 'featlens material:_category',
 'featplace of origin_category',
 'featinterior pockets_category',
 'featitem style_category',
 'featfits brands_category',
 'feathas adaptive lenses_category',
 'featwash_category',
 'featshade_category',
 'featanimal type_category',
 'featlining material_category',
 'featwatch style_category',
 'featgold karat_category',
 'feattemple length_category',
 'feattype of sole

In [204]:
# poniżej usunelismy 'featcolor_category' oraz 'featmanufacturer part number_category' 
# w kolejnym kroku wstawilismy dodane wyzej 'featsport_category' i 'featstyle_category'
# kolejny krok to dodanie 'featmetal type_category' i 'featshape_category' (z powyższej listy cech)
feats = ['brand_category', 'featmetal type_category', 'featshape_category','featbrand_category', 'featgender_category', 'featmaterial_category', 'featstyle_category', 'featsport_category']
# feats += feats_category
# feats = list(set(feats))
run_model(feats, model)

(-57.27903945992567, 4.255887309757601)

In [205]:
X = df[feats].values
y = df['prices_amountmin'].values

m =RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y)
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2572 ± 0.0075,brand_category
0.1046 ± 0.0075,featmaterial_category
0.0253 ± 0.0032,featgender_category
0.0174 ± 0.0015,featbrand_category
0.0128 ± 0.0011,featshape_category
0.0091 ± 0.0014,featmetal type_category
0.0032 ± 0.0012,featstyle_category
0.0001 ± 0.0000,featsport_category


In [194]:
df[df['brand']=='nike'].features_parsed.sample(10).values

array([{'condition': 'new with box', 'type': 'cleats'}, {},
       {'gender': 'men', 'color': 'black', 'condition': 'new with tags'},
       {'sport': 'baseball & softball', 'main color': 'black'},
       {'style': 'athletic sneakers', 'material': 'synthetic', 'condition': 'new with box'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'brand': 'nike', 'color': 'white/metallic silver', 'casual & dress shoe style': 'running shoes', 'shoe width': 'medium (d, m)'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'brand': 'nike', 'color': 'mid navy/sail/gum light brown', 'casual & dress shoe style': 'athletic sneakers', 'shoe width': 'medium (d, m)'},
       {'gender': 'men', 'shoe category': 'mens shoes', 'brand': 'nike', 'color': 'dark obsidian/black/mid navy/black', 'casual & dress shoe style': 'running shoes', 'shoe width': 'medium (d, m)'},
       {'sport': 'soccer', 'country/region of manufacture': 'bosnia and herzegovina', 'type': 'cleats', 'condition': 'new with bo

In [208]:
df.weight.unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [213]:
cp 

ColabDriveGithub.ipynb  day4_meta.ipynb  day5.ipynb  Matrix_DW/
